In [9]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.quantum_info import SparsePauliOp

from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler, Estimator

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# Method to construct the Convolutional Layer for the QCNN
def conv_layer(qc:QuantumCircuit(), para:ParameterVector, iter:int) -> QuantumCircuit():
    start = 0 if qc.num_qubits == 2 else -1
    iter = 3 if iter == 0 else iter
    for i in range(start, qc.num_qubits-1):
        qc.rz(-np.pi / 2, i+1)
        qc.cx(i+1, i)
        qc.rz(para[(3*i)+iter], i)
        qc.ry(para[(3*i)+1+iter], i+1)
        qc.cx(i, i+1)
        qc.ry(para[(3*i)+2+iter], i+1)
        qc.cx(i+1, i)
        qc.rz(np.pi / 2, i)
        qc.barrier()
    if iter == 3:
        iter = 0
    if qc.num_qubits == 2:
        iter += 3
    else: 
        iter += (3*(qc.num_qubits)) 
    return qc, iter

# Method to construct the Pooling Layer for the QCNN
def pool_layer(qc:QuantumCircuit(), para:ParameterVector, iter:int) -> QuantumCircuit():
    half = int(qc.num_qubits/2)
    for i in range(half):
        qc.rz(-np.pi / 2, i+half)
        qc.cx(i+half, i)
        qc.rz(para[(3*i)+iter], i)
        qc.ry(para[(3*i)+1+iter], i+half)
        qc.cx(i, i+half)
        qc.ry(para[(3*i)+2+iter], i+half)
        qc.barrier()
    iter += (half*3)
    return qc, iter

# Method that builds the entire circuit and layers systematically
def qcnn_circuit(num_qubits:int, num_outputs:int) -> QuantumCircuit():
    qc = QuantumCircuit(num_qubits)
    num_layers = np.log2(num_qubits) # Calculates number of convolutional/pooling layers
    
    feature_map = ZFeatureMap(num_qubits)  # Pulls in Qiskit's built in ZFeatureMap
    qc.compose(feature_map, range(num_qubits), inplace=True) # Adds ZFeatureMap to circuit for initialization
    
    parameter_vec = ParameterVector(name="θ", length=int(9*(2**num_layers))-12) # Calculates number of parameters needed
   
    iter = 0 # Tracks where in the Parameter Vector to put the next parameter
    index = num_qubits # Divides itself in half at each layer
    layer = 1 # Tracks the current layer being constructed
    while index > int(num_outputs): # Adds 1 convolution layer and 1 pooling layer then iterates until the desired amount of outputs remain
        conv_qc = QuantumCircuit(index)
        pool_qc = QuantumCircuit(index)
        
        # Adds 1 convolutional layer to the circuit by converting the convolutional circuit to a gate
        conv_qc, iter = conv_layer(conv_qc, parameter_vec, iter)
        qc.compose(conv_qc.to_instruction(label=f'Convolution Layer {layer}'), range(int(num_qubits - index), num_qubits), inplace=True)
        
        # Adds 1 pooling layer to the circuit by converting the pooling circuit to a gate
        pool_qc, iter = pool_layer(pool_qc, parameter_vec, iter)
        qc.compose(pool_qc.to_instruction(label=f'Pooling Layer {layer}'), range(int(num_qubits - index), num_qubits), inplace=True)

        # Handle tracker increment or decrement
        index = int(index / 2)
        layer += 1

    # Constructs weight parameter by removing ZFeatureMap's parameters from the total
    weight_params = qc.parameters - feature_map.parameters
    return qc, feature_map.parameters, weight_params

In [11]:
""" PERSONAL NOTES
2 qubits = 6 parameters
4 qubits = 24 parameters
8 qubtes = 60 parameters
observables=SparsePauliOp.from_list([("Z" + "I" * 7, 1)])
"""

# Declares the Quantum Convolutional Circuit and extracts the input and weigh parameters from the qcnn_circuit method
qcnn, input_params, weight_params = qcnn_circuit(32, 1)

qcnn_est = EstimatorQNN(circuit=qcnn.decompose(), input_params=input_params, weight_params=weight_params)
#qcnn.draw("mpl", style="clifford")

In [12]:
# To plot the progress of the circuits learning
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [13]:
# Declares Classifier
classifier = NeuralNetworkClassifier(
    qcnn_est,
    optimizer=COBYLA(maxiter=500),  # Set max iterations here
    #callback=callback_graph,
    #initial_point=initial_point,
)


** importing training and 

In [14]:
from zipfile import ZipFile

In [15]:
# specifying the zip file name 
file_name = "sonar_data.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    zip.extractall() 

File Name                                             Modified             Size
sonar.all-data                                 2023-12-02 19:14:14        87776
sonar.mines                                    2023-12-02 19:14:14        49217
sonar.rocks                                    2023-12-02 19:14:14        43052
Index                                          2023-12-02 19:14:14          178
sonar.names                                    2023-12-02 19:14:14         5872


In [17]:
#loading data
data = np.genfromtxt("sonar.all-data", delimiter=',', dtype=str)

#splitting the data to features and labels
features = data[:, :-1].astype(float)  
labels = data[:, -1]     

num_rows = features.shape[0]  

zeros_columns = np.zeros((num_rows, 4))

features_with_zeros = np.concatenate((features, zeros_columns), axis=1)

print("Features shape:", features_with_zeros.shape)
print("Labels shape:", labels.shape)

features_with_zeros_list = [np.array(row) for row in features_with_zeros]
features_with_zeros_list = [row[:32] for row in features_with_zeros_list]


#mapping mine to -1 and rock to +1
labels_mapped = np.where(labels == 'M', -1, 1)

Features shape: (208, 64)
Labels shape: (208,)


In [18]:
trainer, tester, trainer_ans, tester_ans = train_test_split(features_with_zeros_list, labels_mapped, test_size=0.25, random_state=246)

In [19]:
x2 = np.asarray(trainer)
y2 = np.asarray(trainer_ans)

print(x2)
print(y2)

objective_func_vals2 = []

plt.rcParams["figure.figsize"] = (12, 6)

[[0.02   0.0371 0.0428 ... 0.3857 0.1307 0.2604]
 [0.0047 0.0059 0.008  ... 0.9151 0.8828 0.8086]
 [0.0378 0.0318 0.0423 ... 0.4831 0.4729 0.3318]
 ...
 [0.0968 0.0821 0.0629 ... 0.5213 0.2316 0.3335]
 [0.0107 0.0453 0.0289 ... 1.     0.9308 0.8478]
 [0.0163 0.0198 0.0202 ... 0.7405 0.8069 0.842 ]]
[ 1 -1  1  1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1
  1 -1  1 -1  1 -1  1  1  1  1 -1  1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1
 -1  1  1  1 -1 -1  1  1 -1 -1  1 -1  1 -1  1 -1  1  1  1 -1 -1  1 -1 -1
  1  1 -1 -1 -1  1 -1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1  1  1  1
  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1  1 -1 -1  1 -1
  1  1  1 -1  1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1  1 -1  1  1 -1  1  1 -1
 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1 -1]


In [ ]:
classifier.fit(x2, y2)